In [17]:
import pandas as pd
from glob import glob

In [2]:
main = pd.read_csv("../dataset/combined.csv")

In [3]:
comm = set(main["community"].to_list())

In [4]:
print(len(comm))

466


In [12]:

for c in comm:
    if c == "other":
        dff = main.query("community == 'other'")
        dff = dff.drop(["community"], axis=1)
        dff.to_csv("./dataset/other.csv", index=False)
        continue
    dff = main.query("community == \'{}\'".format(c))
    dff = dff.drop(["community"], axis=1)
    dff.to_csv("./dataset/{}.csv".format(c), index=False)
    # break


# add edge component

In [43]:
all_nodes = [i for i in range(7015)]

In [16]:
edgeOrdering = [0]*7015

In [20]:
files = glob("dataset\\*")
files.pop() # drop the other option

'dataset\\other.csv'

In [21]:
from collections import Counter

In [26]:
for file in files:
    df = pd.read_csv(file)
    nodes = df['node1'].to_list()
    count = dict(Counter(nodes))
    ordering = []
    for i in count:
        ordering.append([count[i], i])
    ordering.sort(reverse=True)
    c = 0
    for pair in ordering:
        edgeOrdering[pair[1]] = c
        c += 1
    # break

In [27]:
print(edgeOrdering)

[1, 0, 26, 2, 25, 6, 19, 12, 18, 17, 24, 5, 31, 30, 16, 15, 4, 11, 10, 9, 0, 8, 29, 1, 14, 13, 28, 3, 23, 22, 21, 20, 27, 7, 4, 0, 2, 18, 26, 1, 17, 28, 25, 6, 16, 5, 15, 14, 13, 24, 23, 12, 3, 11, 22, 10, 21, 20, 19, 9, 8, 7, 27, 1, 0, 3, 2, 1, 0, 0, 7, 6, 5, 3, 2, 1, 4, 1, 0, 1, 0, 2, 1, 0, 3, 2, 1, 0, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 0, 1, 0, 5, 16, 6, 0, 10, 170, 61, 169, 168, 60, 76, 15, 94, 43, 12, 7, 300, 42, 59, 231, 299, 298, 11, 167, 58, 166, 127, 267, 126, 144, 143, 142, 141, 140, 139, 138, 2, 3, 297, 9, 1, 75, 208, 4, 74, 93, 207, 36, 8, 26, 230, 13, 165, 266, 296, 14, 265, 264, 263, 57, 295, 206, 205, 17, 262, 261, 21, 229, 204, 41, 56, 203, 164, 125, 27, 294, 293, 202, 201, 200, 292, 291, 290, 199, 124, 163, 162, 161, 160, 159, 35, 198, 92, 40, 260, 259, 258, 31, 30, 137, 197, 20, 19, 228, 227, 257, 123, 136, 135, 256, 255, 65, 122, 121, 120, 23, 119, 118, 117, 73, 289, 288, 25, 116, 115, 114, 113, 112, 254, 253, 34, 196, 287, 39, 158, 157, 91, 72, 226, 38

# temporal ordering

In [28]:
temporalOrdering = [0]*7015

In [34]:
for file in files[:]:
    df = pd.read_csv(file)
    df = df.sort_values(by=['temporal_similarity', 'topic_similarity'], ascending=False)
    d = set()
    c = 0
    for node in df['node1'].to_list():
        if node not in d:
            temporalOrdering[node] = c
            c += 1
            d.add(node)
    # break

In [35]:
print(temporalOrdering)

[0, 1, 6, 4, 26, 1, 24, 16, 13, 14, 21, 10, 15, 5, 30, 31, 3, 28, 22, 19, 17, 18, 25, 2, 23, 7, 20, 0, 8, 9, 11, 12, 29, 27, 5, 1, 9, 6, 10, 12, 15, 27, 25, 22, 28, 3, 18, 7, 8, 24, 26, 11, 2, 16, 4, 14, 13, 19, 17, 0, 20, 21, 23, 0, 1, 0, 2, 1, 3, 2, 3, 7, 5, 0, 1, 4, 6, 0, 1, 0, 1, 0, 3, 1, 2, 2, 0, 1, 4, 5, 13, 0, 6, 10, 9, 7, 1, 12, 11, 8, 2, 3, 0, 1, 0, 1, 45, 188, 75, 10, 14, 222, 172, 160, 28, 64, 203, 74, 76, 83, 41, 82, 299, 173, 159, 210, 52, 53, 1, 15, 16, 72, 168, 178, 239, 9, 65, 25, 11, 79, 80, 175, 84, 67, 271, 118, 54, 213, 162, 26, 240, 177, 164, 165, 92, 136, 223, 97, 132, 158, 298, 81, 238, 264, 179, 182, 114, 237, 241, 127, 282, 205, 47, 206, 207, 89, 71, 279, 157, 86, 99, 289, 290, 27, 50, 51, 270, 98, 70, 233, 124, 201, 22, 23, 24, 113, 147, 34, 35, 36, 0, 2, 242, 120, 3, 180, 243, 244, 245, 230, 226, 224, 46, 40, 151, 7, 8, 192, 140, 142, 262, 130, 235, 141, 148, 149, 134, 135, 66, 193, 33, 31, 85, 32, 263, 246, 154, 292, 266, 143, 163, 166, 87, 48, 284, 208, 198

# temporal ordering

In [37]:
topicOrdering = [0]*7015

In [38]:
for file in files[:]:
    df = pd.read_csv(file)
    df = df.sort_values(by=['topic_similarity', 'temporal_similarity'], ascending=False)
    d = set()
    c = 0
    for node in df['node1'].to_list():
        if node not in d:
            topicOrdering[node] = c
            c += 1
            d.add(node)
    # break

In [39]:
print(topicOrdering)

[0, 1, 16, 6, 31, 1, 19, 20, 26, 27, 23, 13, 28, 15, 7, 3, 4, 10, 12, 22, 2, 21, 30, 9, 11, 5, 29, 0, 17, 18, 24, 25, 14, 8, 1, 6, 0, 14, 13, 2, 22, 25, 9, 5, 18, 19, 10, 3, 4, 7, 17, 16, 11, 23, 20, 21, 12, 8, 24, 15, 26, 27, 28, 0, 1, 0, 2, 1, 3, 4, 5, 3, 7, 0, 1, 2, 6, 0, 1, 0, 1, 0, 2, 1, 3, 2, 0, 1, 8, 9, 11, 6, 3, 10, 7, 5, 0, 2, 1, 12, 13, 4, 0, 1, 0, 1, 128, 27, 158, 49, 180, 222, 81, 121, 48, 186, 134, 207, 210, 198, 84, 78, 202, 133, 113, 228, 230, 229, 1, 82, 50, 51, 7, 8, 289, 69, 119, 72, 71, 70, 114, 85, 56, 47, 196, 54, 204, 44, 45, 137, 288, 46, 140, 40, 11, 238, 218, 66, 86, 129, 112, 36, 286, 31, 9, 18, 232, 293, 294, 15, 290, 208, 99, 212, 213, 52, 59, 268, 145, 168, 118, 199, 200, 237, 278, 192, 194, 58, 165, 179, 191, 181, 292, 161, 162, 57, 125, 100, 10, 14, 0, 2, 295, 12, 3, 16, 281, 109, 110, 203, 277, 271, 234, 83, 65, 126, 188, 87, 106, 107, 164, 269, 197, 176, 53, 153, 241, 251, 63, 138, 220, 193, 150, 139, 282, 296, 13, 62, 76, 61, 55, 60, 38, 149, 42, 43, 7

In [44]:
df = pd.DataFrame(data={"node": all_nodes,
                        "edgeOrdering": edgeOrdering,
                        "topicOrdering": topicOrdering,
                        "temporalOrdering": temporalOrdering})

In [45]:
df.to_csv("seriation.csv", index=False)

# community within it

In [52]:
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx

In [53]:
communityOrdering = [0]*7015

In [57]:
kool = 1
for file in files:
    print(kool)
    kool+=1
    G = nx.Graph()
    dff = pd.read_csv(file)
    # G.add_nodes_from(set(dff['node1'].to_list()))
    for ind in dff.index:
        G.add_edge(dff["node1"][ind], dff["node2"][ind])
    
    partition = nx.community.louvain_communities(G)
    # print(partition)
    c = 0
    for part in partition:
        for i in part:
            communityOrdering[i] = c
            c += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [60]:
df["communityOrdering"] = communityOrdering
df.to_csv("seriation.csv", index=False)